In [1]:
#add login to .gitignore file
#solve for if link contains a word with no space like 
#detect new posts on https://forums.pelicanparts.com/porsche-cars-sale/

In [2]:
#import the required packages/libs
from craigslist import CraigslistForSale
import pandas as pd
import webbrowser
import smtplib
import time
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [3]:
already_checked = [] #hold the list of already emailed listings
temp = [] #store the list of urls to later be added to already_checked
email_msg = [] #store the set of listings from the search to be emailed at the end of the loops of calls

#read in existing postings
f = open("searched.txt", "r")
already_checked = f.readlines()
f.close()
# remove whitespace characters like `\n` at the end of each line
already_checked = [x.strip() for x in already_checked] 


#define the send email function, so that lists of results can be sent via email
def send_email(body, sender, recipients, subject):
    msg = 'Subject: {}\n\n{}'.format(subject, body)
    smtpserver = 'smtp.gmail.com:587'
    try:
        server = smtplib.SMTP(smtpserver)
        server.ehlo()
        server.starttls()
        server.login('thefifthfox0@gmail.com', 'computerfox')
        server.sendmail(sender, recipients, msg)
    except Exception as e:
        print(e)

#Search craigslist for cars and perform the necessary functions based on the parameters provided
def search_craigslist_for_cars(loc,car,minyear,maxyear,minprice,maxprice,openlinks,email,printheader):
    
    #Declare some initial variables
    msg = [] #print the set of listings returned by a given call of search_craiglists for cards
    #already_checked = ['https://anchorage.craigslist.org/cto/d/1978-porsche-911-sc-only-5000/6680836329.html','https://austin.craigslist.org/cto/d/1976-porsche-911-convertible/6664202810.html','https://bakersfield.craigslist.org/cto/d/911-porsche-convertible/6686260433.html','https://bham.craigslist.org/cto/d/1983-porsche-911-bridgestone/6687907347.html','https://boise.craigslist.org/cto/d/rare-1970-porsche-911-body/6649790854.html','https://charleston.craigslist.org/cto/d/1986-porsche-911-conv/6649734485.html','https://cleveland.craigslist.org/cto/d/1983-porsche-911-goodyear/6687914825.html','https://columbia.craigslist.org/cto/d/1983-burgundy-porsche-911/6687196348.html','https://daytona.craigslist.org/cto/d/1986-porsche-911-carrera-conv/6649761594.html','https://eastoregon.craigslist.org/cto/d/early-porsche-911-project-wtb/6679741734.html','https://fargo.craigslist.org/cto/d/1983-porsche-911-garaged-with/6682299789.html','https://flint.craigslist.org/cto/d/1983-porsche-911-burgundy/6687285576.html','https://grandrapids.craigslist.org/cto/d/porsche-911-project-or-fixer/6685967106.html','https://greenville.craigslist.org/cto/d/1986-porsche-911-conv/6649701036.html','https://houston.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678163728.html','https://hudsonvalley.craigslist.org/cto/d/1982-porsche-911-slant-nose/6681725992.html','https://inlandempire.craigslist.org/cto/d/1983-porsche-911-driver-airbag/6687979723.html','https://lansing.craigslist.org/cto/d/911-porsche-looking-for-older/6687104481.html','https://lasvegas.craigslist.org/cto/d/1987-porsche-911-carrera/6687002558.html','https://lasvegas.craigslist.org/cto/d/1969-porsche-911-baja-race/6668904623.html','https://lasvegas.craigslist.org/cto/d/1977-porsche-911-sc-targa/6680151081.html','https://monterey.craigslist.org/cto/d/red-1977-porsche-911-sc-targa/6680149748.html','https://monterey.craigslist.org/ctd/d/1984-porsche-911-cab-guards/6677467366.html','https://nmi.craigslist.org/cto/d/69-porsche-911-irish-green/6656476893.html','https://norfolk.craigslist.org/cto/d/1972-porsche-911/6663694575.html','https://orlando.craigslist.org/cto/d/1991-porsche-911-carrera-2/6680295712.html','https://pensacola.craigslist.org/cto/d/1983-porsche-911-manual/6684020499.html','https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678187037.html','https://philadelphia.craigslist.org/cto/d/1974-porsche-911-targa/6670015872.html','https://reno.craigslist.org/cto/d/porsche-911/6683051897.html','https://sacramento.craigslist.org/cto/d/1967-porsche-911/6682711263.html']
    
    #Pull data based on parameters
    cars = CraigslistForSale(site=loc, area='', category='cars-trucks', 
                         filters={'min_price': minprice, 'max_price': maxprice, 'min_year':minyear, 'max_year':maxyear,'query':car})
    
    #Print title if desired
    if printheader == True:
        header = ''
        header = str(cars.filters['min_auto_year']) + ' till ' + str(cars.filters['max_auto_year']) + ' ' +cars.filters['query'] + ' between ' + str(cars.filters['min_price']) + ' and ' + str(cars.filters['max_price']) + ' in the ' + loc + ' area '
        print(header)
    
    #Loop through the results
    for x in cars.get_results():
        #Only return results if url contains one of the words from the search critera AND
        #Also only return results that are not in the already_checked list
        if (len(set(car.split(' ')) & set( x['url'].split('/')[5].split('-'))) > 1) and ((x['url'] in already_checked) == False):
            msg.append('Posted: {} Price: {} Link: {}'.format(x['datetime'], x['price'], x['url']))
            email_msg.append('Posted: {} Price: {} Link: {}'.format(x['datetime'], x['price'], x['url'])) 
            #Temporary holding of URL from the search results as a list to use as filter against the already_checked list
            temp.append(x['url'])
            
            #Open urls in webbrowser if desired
            if openlinks == True:
                webbrowser.open(x['url'])
    
    #Send email with search results if desired and search results exist
    if email == True and len(msg) > 0:
        send_email('\n'.join(msg), 'thefifthfox0@gmail.com', ['Qman2386@gmail.com'], header)
    
    #Print out the search results to the console 
    for _ in msg:
        print(_)

In [4]:
#search_craigslist_for_cars('philadelphia','porsche 911',1900,1998,0,30000,False,False,True) 
#search_craigslist_for_cars('philadelphia','porsche 911',1900,1998,'','',False,False,True) 
#search_craigslist_for_cars('philadelphia','nissan skyline',1900,2018,'','',True,False,True)
#search_craigslist_for_cars('philadelphia','toyota corolla',1986,1987,'','',False,False,True)
#,'impreza','mazda3','tsx','gti','accord','miata','corolla','fit','sonic']

In [5]:
cars = ['quigley']
sites = ['abilene','akroncanton','albany','albanyga','albuquerque','allentown','altoona','amarillo','ames','anchorage','annapolis','annarbor','appleton','asheville','ashtabula','athensga','athensohio','atlanta','auburn','augusta','austin','bakersfield','baltimore','batonrouge','battlecreek','beaumont','bellingham','bemidji','bend','bgky','bham','bigbend','billings','binghamton','bismarck','blacksburg','bloomington','bn','boise','boone','boston','boulder','bozeman','brainerd','brownsville','brunswick','buffalo','butte','capecod','carbondale','catskills','cedarrapids','cenla','centralmich','cfl','chambana','chambersburg','charleston','charlestonwv','charlotte','charlottesville','chattanooga','chautauqua','chicago','chico','chillicothe','cincinnati','clarksville','cleveland','clovis','cnj','collegestation','columbia','columbiamo','columbus','columbusga','cookeville','corpuschristi','corvallis','cosprings','csd','dallas','danville','dayton','daytona','decatur','delaware','delrio','denver','desmoines','detroit','dothan','dubuque','duluth','eastco','easternshore','eastidaho','eastky','eastnc','eastoregon','easttexas','eauclaire','elko','elmira','elpaso','enid','erie','eugene','evansville','fairbanks','fargo','farmington','fayar','fayetteville','fingerlakes','flagstaff','flint','florencesc','fortcollins','fortdodge','fortlauderdale','fortmyers','fortsmith','fortwayne','frederick','fredericksburg','fresno','gadsden','gainesville','galveston','glensfalls','goldcountry','grandforks','grandisland','grandrapids','greatfalls','greenbay','greensboro','greenville','gulfport','hanford','harrisburg','harrisonburg','hartford','hattiesburg','helena','hickory','hiltonhead','holland','honolulu','houma','houston','hudsonvalley','humboldt','huntington','huntsville','miami','imperial','indianapolis','inlandempire','iowacity','ithaca','jackson','jacksontn','jacksonville','janesville','jerseyshore','jonesboro','joplin','juneau','jxn','kalamazoo','kalispell','kansascity','kenai','keys','killeen','kirksville','klamath','knoxville','kokomo','kpr','ksu','lacrosse','lafayette','lakecharles','lakecity','lakeland','lancaster','lansing','laredo','lasalle','lascruces','lasvegas','lawrence','lawton','lewiston','lexington','limaohio','lincoln','littlerock','logan','longisland','losangeles','louisville','loz','lubbock','lynchburg','macon','madison','maine','mankato','mansfield','marshall','martinsburg','masoncity','mattoon','mcallen','meadville','medford','memphis','mendocino','merced','meridian','miami','milwaukee','minneapolis','missoula','mobile','modesto','mohave','monroe','monroemi','montana','monterey','montgomery','morgantown','moseslake','muncie','muskegon','myrtlebeach','nacogdoches','nashville','natchez','nd','nesd','newhaven','newjersey','newlondon','neworleans','newyork','nh','nmi','norfolk','northernwi','northmiss','northplatte','nwct','nwga','nwks','ocala','odessa','ogden','okaloosa','oklahomacity','olympic','omaha','oneonta','onslow','orangecounty','oregoncoast','orlando','ottumwa','outerbanks','owensboro','palmsprings','panamacity','parkersburg','pennstate','pensacola','peoria','philadelphia','phoenix','pittsburgh','plattsburgh','poconos','porthuron','portland','potsdam','prescott','providence','provo','pueblo','pullman','quadcities','quincy','racine','raleigh','rapidcity','reading','redding','reno','richmond','richmondin','rmn','roanoke','rochester','rockford','rockies','roseburg','roswell','sacramento','saginaw','salem','salina','saltlakecity','sanangelo','sanantonio','sandiego','sandusky','sanmarcos','santabarbara','santafe','santamaria','sarasota','savannah','scottsbluff','scranton','sd','seattle','seks','semo','sfbay','sheboygan','shoals','showlow','shreveport','sierravista','siouxcity','siouxfalls','siskiyou','skagit','slo','smd','southbend','southcoast','southjersey','spacecoast','spokane','springfield','springfieldil','statesboro','staugustine','stcloud','stgeorge','stillwater','stjoseph','stlouis','stockton','susanville','swks','swmi','swv','swva','syracuse','tallahassee','tampa','terrehaute','texarkana','texoma','thumb','tippecanoe','toledo','topeka','treasure','tricities','tucson','tulsa','tuscaloosa','tuscarawas','twinfalls','twintiers','up','utica','valdosta','ventura','vermont','victoriatx','visalia','waco','washingtondc','waterloo','watertown','wausau','wenatchee','westernmass','westky','westmd','westslope','wheeling','wichita','wichitafalls','williamsport','wilmington','winchester','winstonsalem','worcester','wv','wyoming','yakima','york','youngstown','yubasutter','yuma','zanesville']

In [ ]:
search_craigslist_for_cars('philadelphia','honda',1900,2018,0,100000,False,False,True)

1900 till 2018 honda between 0 and 100000 in the philadelphia area 


In [ ]:
 cars = CraigslistForSale(site='philadelphia', area='', category='cars-trucks', 
                         filters={'min_price': 0, 'max_price': 100000, 'min_year':1900, 'max_year':2018,'query':'honda'})
    
for x in cars.get_results():
    print(x)

{'id': '6720301342', 'repost_of': None, 'name': '2008 Honda CR-V Best Price For Sale', 'url': 'https://philadelphia.craigslist.org/cto/d/2008-honda-cr-best-price-for/6720301342.html', 'datetime': '2018-10-14 09:43', 'price': '$2000', 'where': None, 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6723014863', 'repost_of': '6720536056', 'name': 'Honda CR-V,1997,AWD, 128k,runs great new pa inspection', 'url': 'https://philadelphia.craigslist.org/cto/d/honda-cr-v1997awd-128kruns/6723014863.html', 'datetime': '2018-10-14 09:37', 'price': '$1699', 'where': 'Folcroft', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6722999924', 'repost_of': '3107020476', 'name': 'HONDA ACCORD  - SUPER SLEAN - LIKE NEW - LOW MILES', 'url': 'https://philadelphia.craigslist.org/cto/d/honda-accord-super-slean-like/6722999924.html', 'datetime': '2018-10-14 09:13', 'price': '$2995', 'where': 'NE PHILLY 19115', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6722990772', 'repost_

{'id': '6722202025', 'repost_of': None, 'name': '2015 Honda CR-V CRV LX Sport Utility 4D suv Black - FINANCE ONLINE', 'url': 'https://philadelphia.craigslist.org/ctd/d/2015-honda-cr-crv-lx-sport/6722202025.html', 'datetime': '2018-10-13 07:05', 'price': '$17700', 'where': 'CALL 215-995-2743 FOR CUSTOM PAYMENT', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6722157540', 'repost_of': '6421966686', 'name': '2015 Honda CRF450R', 'url': 'https://philadelphia.craigslist.org/mcy/d/2015-honda-crf450r/6722157540.html', 'datetime': '2018-10-13 01:45', 'price': '$4199', 'where': 'Bucks County', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6704925125', 'repost_of': None, 'name': '1995 Honda Accord LX', 'url': 'https://philadelphia.craigslist.org/cto/d/1995-honda-accord-lx/6704925125.html', 'datetime': '2018-10-13 01:11', 'price': '$1500', 'where': '21st & Allegheny', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6707667574', 'repost_of': '5659067479', 'na

{'id': '6716315119', 'repost_of': '6696396346', 'name': '2009 Honda Fit', 'url': 'https://philadelphia.craigslist.org/cto/d/2009-honda-fit/6716315119.html', 'datetime': '2018-10-12 08:59', 'price': '$4999', 'where': 'Philadelphia', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6718934805', 'repost_of': None, 'name': 'honda accord ex 4 door automatic', 'url': 'https://philadelphia.craigslist.org/cto/d/honda-accord-ex-4-door/6718934805.html', 'datetime': '2018-10-12 08:49', 'price': '$1500', 'where': None, 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6721349220', 'repost_of': '6690561545', 'name': '2003 HONDA CRV', 'url': 'https://philadelphia.craigslist.org/cto/d/2003-honda-crv/6721349220.html', 'datetime': '2018-10-12 08:48', 'price': '$4999', 'where': '19095', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6701445438', 'repost_of': None, 'name': '1997 Honda CRV', 'url': 'https://philadelphia.craigslist.org/cto/d/1997-honda-crv/6701445438.html'

{'id': '6716626428', 'repost_of': None, 'name': '2014 HONDA ACCORD', 'url': 'https://philadelphia.craigslist.org/cto/d/2014-honda-accord/6716626428.html', 'datetime': '2018-10-11 07:50', 'price': '$11000', 'where': 'Elkins Park', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6716629225', 'repost_of': None, 'name': '2014 Honda Accord 23K', 'url': 'https://philadelphia.craigslist.org/cto/d/2014-honda-accord-23k/6716629225.html', 'datetime': '2018-10-11 07:50', 'price': '$11000', 'where': 'Elkins Park', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6707384726', 'repost_of': '6235424453', 'name': 'Honda  Accord CVCC 1st generation 1977', 'url': 'https://philadelphia.craigslist.org/cto/d/honda-accord-cvcc-1st/6707384726.html', 'datetime': '2018-10-11 07:41', 'price': '$9999', 'where': 'NE Philly', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6701214499', 'repost_of': '6582146242', 'name': '2011 Honda Crz automatic  Hybrid  low 53k miles', 'url': 'h

{'id': '6719024027', 'repost_of': None, 'name': '2006 Honda Accord', 'url': 'https://philadelphia.craigslist.org/cto/d/2006-honda-accord/6719024027.html', 'datetime': '2018-10-09 13:37', 'price': '$700', 'where': 'Levittown', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6718945385', 'repost_of': None, 'name': 'power/folder#4WD-mirrors2004 Honda Accord ComfortV6Access', 'url': 'https://philadelphia.craigslist.org/cto/d/power-folder4wd-mirrors2004/6718945385.html', 'datetime': '2018-10-09 13:35', 'price': '$1000', 'where': None, 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6704316857', 'repost_of': '6609891210', 'name': '2006 Honda Accord For Sale', 'url': 'https://philadelphia.craigslist.org/cto/d/2006-honda-accord-for-sale/6704316857.html', 'datetime': '2018-10-09 13:26', 'price': '$4700', 'where': 'Northeast Phila', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6707851432', 'repost_of': None, 'name': '** HONDA ACCORD 2004 - LEATHER - AUTOMAT

...

{'id': '6698397575', 'repost_of': '6343776857', 'name': '2003 Honda Accord ex sedan', 'url': 'https://philadelphia.craigslist.org/cto/d/2003-honda-accord-ex-sedan/6698397575.html', 'datetime': '2018-09-15 09:00', 'price': '$2200', 'where': 'Delran, NJ', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6679123586', 'repost_of': None, 'name': 'Honda Shadow Sabre VT1100 - Motorcycle', 'url': 'https://philadelphia.craigslist.org/mcy/d/honda-shadow-sabre-vt1100/6679123586.html', 'datetime': '2018-09-15 08:48', 'price': '$3900', 'where': 'Norristown', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6698376725', 'repost_of': None, 'name': '2006 Honda Pilot EX w/Leather and Navigation', 'url': 'https://philadelphia.craigslist.org/ctd/d/2006-honda-pilot-ex-leather/6698376725.html', 'datetime': '2018-09-15 08:15', 'price': '$5495', 'where': '*2006* *Honda* *Pilot* *EX* *w/Leather* *and* *Nav', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6698371962', 'repost

{'id': '6720585298', 'repost_of': '6697166394', 'name': '2010 Mercedes-Benz ML 350 4MATIC*Silver* MINT Condition *We Finance*', 'url': 'https://philadelphia.craigslist.org/ctd/d/2010-mercedes-benz-ml-350/6720585298.html', 'datetime': '2018-10-11 10:38', 'price': '$12999', 'where': 'Exit 8 NJ TURNPIKE Route 33 Monroe, NJ', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6720563347', 'repost_of': '6696303069', 'name': '2009 BMW X5 xDrive35d * Diesel * Navi * Panoramic Roof *  Finance NOW!', 'url': 'https://philadelphia.craigslist.org/ctd/d/2009-bmw-x5-xdrive35d-diesel/6720563347.html', 'datetime': '2018-10-11 10:15', 'price': '$8799', 'where': '6 Bentley Road, Monroe, NJ', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6720553517', 'repost_of': '6682530251', 'name': '2008 Volvo s40', 'url': 'https://philadelphia.craigslist.org/cto/d/2008-volvo-s40/6720553517.html', 'datetime': '2018-10-11 10:08', 'price': '$4000', 'where': 'ne', 'has_image': True, 'has_map': True

{'id': '6714743894', 'repost_of': None, 'name': 'Ford Fusion Philadelphia Buy Here Pay Here bad No Credit Check', 'url': 'https://philadelphia.craigslist.org/ctd/d/ford-fusion-philadelphia-buy/6714743894.html', 'datetime': '2018-10-04 12:01', 'price': '$12900', 'where': 'from $500 Down - Only Liability Insurance Required', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6714679142', 'repost_of': None, 'name': 'Hyundai Sonata Buy Here Pay Here - 5 Min bad No Credit Check', 'url': 'https://philadelphia.craigslist.org/ctd/d/hyundai-sonata-buy-here-pay/6714679142.html', 'datetime': '2018-10-04 11:01', 'price': '$12900', 'where': 'from $500 Down - Only Liability Insurance Required', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6714649398', 'repost_of': None, 'name': '1967 Ford F250 Custom Cab', 'url': 'https://philadelphia.craigslist.org/cto/d/1967-ford-f250-custom-cab/6714649398.html', 'datetime': '2018-10-04 10:35', 'price': '$5700', 'where': 'Miami', 'has_image

{'id': '6707813421', 'repost_of': '4963230398', 'name': '2007 CHEV TRAILBLZER 4X4 CLEAN RUNS GREAT SALE!!!', 'url': 'https://philadelphia.craigslist.org/ctd/d/2007-chev-trailblzer-4x4/6707813421.html', 'datetime': '2018-09-26 09:25', 'price': '$2950', 'where': "ROBINS SUV'S 2ND ST AND ERIE AVE PHILA", 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6707811677', 'repost_of': '6609396483', 'name': '2007 DODGE RAM P/UP WARRANTY  sale   clean', 'url': 'https://philadelphia.craigslist.org/ctd/d/2007-dodge-ram-up-warranty/6707811677.html', 'datetime': '2018-09-26 09:23', 'price': '$3950', 'where': 'ROBINS SUVS AND TRUCKS 2ND ST & ERIE', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6707812658', 'repost_of': '5161778802', 'name': '2004 SPORT TRAC PICKUP TRUCK 4X4', 'url': 'https://philadelphia.craigslist.org/ctd/d/2004-sport-trac-pickup-truck/6707812658.html', 'datetime': '2018-09-26 09:24', 'price': '$4950', 'where': 'robins trucks 3701 n 2nd st phila', 'has_image':

{'id': '6698695845', 'repost_of': None, 'name': 'Chevrolet Equinox Buy Buy Here Pay Here - bad / no credit check', 'url': 'https://philadelphia.craigslist.org/ctd/d/chevrolet-equinox-buy-buy/6698695845.html', 'datetime': '2018-09-15 14:01', 'price': '$12900', 'where': 'from $500 Down - Only Liability Insurance Required', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6698643589', 'repost_of': '6639296694', 'name': 'YOU CAN DRIVE THIS CAR WITH A $500 DOWN PAYMENT ALL CREDIT ACCEPTED', 'url': 'https://philadelphia.craigslist.org/ctd/d/you-can-drive-this-car-with/6698643589.html', 'datetime': '2018-09-15 13:13', 'price': '$500', 'where': 'EVERYONE IS APPROVED!!!!!!', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6698629500', 'repost_of': None, 'name': 'Hyundai Elantra Philadelphia Buy Here Pay Here bad no credit check', 'url': 'https://philadelphia.craigslist.org/ctd/d/hyundai-elantra-philadelphia/6698629500.html', 'datetime': '2018-09-15 13:01', 'price': '$500'

In [ ]:
#Search all sites for data
start = time.clock()
for y in cars:
    for x in sites:
        search_craigslist_for_cars(x,y,1900,2018,10,100000,False,False,True) 
end = time.clock()
print("It took approx " + color.BOLD + str(round((end-start)/60)) + " minutes " + color.END + "to run this program")

#append postings to the file
with open("searched.txt", 'a+') as file_handler:
    for item in temp:
        file_handler.write("{}\n".format(item))

#send email with 
send_email('\n'.join(email_msg), 'thefifthfox0@gmail.com', ['Qman2386@gmail.com'], '2016 till 2019 toyota camry between 0 and 15000')